Demo for the new plotting functions of `BioPsyKit`: `bp.plotting.feature_boxplot()` and `bp.plotting.multi_feature_boxplot()` and the derived functions: `bp.protocols.plotting.saliva_feature_boxplot()` and `bp.protocols.plotting.saliva_multi_feature_boxplot()`.

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from biopsykit.utils.dataframe_handling import multi_xs
from biopsykit.stats import StatsPipeline
from biopsykit.protocols import CAR

from carwatch_analysis.saliva_helper import import_cortisol_raw, import_cortisol_features, analysis_saliva_raw, analysis_saliva_features
from carwatch_analysis.general_helper import describe_groups_df

from statannot import add_stat_annotation

import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
pg.options['round'] = 3

sns.set(style='ticks', context='talk')
#plt.rcParams['figure.figsize'] = (15,5)
plt.rcParams['figure.figsize'] = (10,5)
plt.close('all')

export = True

palette = bp.colors.fau_palette
sns.set_palette(palette)
palette

In [ ]:
export_path = Path("../../exports")
plot_path = export_path.joinpath("plots")
stats_path = export_path.joinpath("statistics")

paper_path = Path("/Users/Richer/Documents/PhD/Research/Papers/2021_CARInnerClock_EMBC/img")
#paper_path = paper_path.joinpath('img')

bp.utils.file_handling.mkdirs([plot_path, stats_path, paper_path])

## Data Import

In [ ]:
car = CAR(saliva_times=[i*15 for i in range(5)])

condition_order = ["Spontaneous", "Known Alarm", "Unknown Alarm"]

### Raw Cortisol

In [ ]:
cort_samples = import_cortisol_raw(export_path.joinpath("cortisol_samples_cleaned.csv"))
cort_samples.head()

### Cortsol Features

In [ ]:
cort_features = import_cortisol_features(export_path.joinpath("cortisol_features_cleaned.csv"))
cort_features.head()

## MIXED ANOVA

In [ ]:
pipeline = StatsPipeline(
    steps=[
        ('prep', 'normality'),
        ('prep', 'equal_var'),
        ('test', 'mixed_anova'),
        ('posthoc', 'pairwise_ttests')
    ],
    params={
        'dv': 'cortisol',
        'between': "condition",
        'within': 'sample',
        'subject': 'night_id',
        #'padjust': 'bonf'
    }
)

pipeline.apply(cort_samples)
pipeline.results_cat("posthoc")
#pipeline.display_results()

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets("posthoc", "interaction", "multi")

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=cort_samples, 
    x='sample', 
    y='cortisol', 
    hue='condition',
    hue_order=condition_order, 
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax)
fig.tight_layout()

## RM ANOVA

In [ ]:
pipeline = StatsPipeline(
    steps=[
        ('prep', 'normality'),
        ('prep', 'equal_var'),
        ('test', 'rm_anova'),
        ('posthoc', 'pairwise_ttests')
    ],
    params={
        'dv': 'cortisol',
        'within': 'sample',
        'subject': 'night_id',
        'padjust': 'bonf'
    }
)

pipeline.apply(cort_samples)
#pipeline.display_results()
pipeline.results_cat("posthoc")

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets("posthoc", "within", "single")

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=cort_samples, 
    x='sample', 
    y='cortisol', 
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax)
fig.tight_layout()

## ANOVA

In [ ]:
pipeline = StatsPipeline(
    steps=[
        ('prep', 'normality'),
        ('prep', 'equal_var'),
        ('test', 'anova'),
        ('posthoc', 'pairwise_ttests')
    ],
    params={
        'dv': 'cortisol',
        'between': "condition",
        'padjust': 'bonf'
    }
)

pipeline.apply(cort_samples)
pipeline.results_cat("posthoc")

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets("posthoc", "between", "single")

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=cort_samples, 
    x='condition', 
    y='cortisol', 
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax)
fig.tight_layout()

In [ ]:
pipeline = StatsPipeline(
    steps=[
        ('prep', 'normality'),
        ('prep', 'equal_var'),
        ('test', 'anova'),
        ('posthoc', 'pairwise_gameshowell')
    ],
    params={
        'groupby': 'biomarker',
        'dv': 'cortisol',
        'between': "condition",
        'padjust': 'bonf'
    }
)

pipeline.apply(cort_features)
pipeline.results_cat("posthoc")

In [ ]:
feature = 'slopeS0S3'

box_pairs, pvals = pipeline.sig_brackets('posthoc', stats_type='between', plot_type='single', features=feature)
data_plot = cort_features.xs(feature, level='biomarker')

fig, ax = plt.subplots()
car.saliva_feature_boxplot(data=data_plot, x='condition', saliva_type='cortisol', feature=feature, order=condition_order, 
                           stats_kwargs={'box_pairs': box_pairs, 'pvalues': pvals}, 
                           ax=ax)
fig.tight_layout()

In [ ]:
feature = ['slopeS0S4']

box_pairs, pvals = pipeline.sig_brackets('posthoc', stats_type='between', plot_type='multi', features=feature, x="biomarker")

data = multi_xs(cort_features, feature, level='biomarker')

fig, ax = plt.subplots()
car.saliva_feature_boxplot(data=data, x='biomarker', saliva_type='cortisol', hue='condition', feature=feature, hue_order=condition_order, 
                           stats_kwargs={'box_pairs': box_pairs, 'pvalues': pvals}, 
                           ax=ax)
fig.tight_layout()

In [ ]:
features = ['slopeS0S3', 'slopeS0S4']

box_pairs, pvals = pipeline.sig_brackets('posthoc', stats_type='between', plot_type='multi', features=features, x="biomarker")

data = cort_features.filter(like='slope', axis=0)

fig, ax = plt.subplots()
car.saliva_feature_boxplot(data=data, x='biomarker', saliva_type='cortisol', hue='condition', hue_order=condition_order, feature=features, 
                           stats_kwargs={'box_pairs': box_pairs, 'pvalues': pvals}, 
                           ax=ax)
fig.tight_layout()